# APIs Lab - AEMET API - Eduardo Arroyo

Importamos librerias de pandas y requests

In [1]:
import pandas as pd
import requests
import folium
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
token = os.getenv("AEMET_APIKEY") 

Hacemos un acopio de los links que vamos a necesitar para el caso de estudio

In [3]:
url_common = "https://opendata.aemet.es/opendata"

url_allstations = url_common + "/api/valores/climatologicos/inventarioestaciones/todasestaciones/"

url_weather_actual_all = url_common + '/api/observacion/convencional/todas'

Definimos las funciones necesarias para la totalidad del ejercicio

In [4]:
def callaemet_url(url):
    '''
    Funcion que obtiene la respuesta de la URL de la api de AEMET que le pases
    '''
    querystring = {"api_key":token}

    headers = {
        'cache-control': "no-cache"
        }

    
    res = requests.request("GET", url, headers=headers, params=querystring)
    
    return res, res.json()

def call_data(url):
    '''
    Funcion que obtiene los datos de la URL de la api de AEMET que le pases. Esto es debido a que la respuesta de las
    urls de la api de aemet responden siempre con una estructura donde en el apartado datos te dan otra url donde te
    dan los datos finales que se estan buscando.
    '''
    res = callaemet_url(url)
    
    final_res = callaemet_url(res[1]["datos"])
    
    return final_res

def simple_dict2df(dict_):
    '''
    funcion que obtiene un diccionario con un solo escalon de valores y lo convierte en df. Es decir el diccionario
    ha de tener una esturcura donde en el esquema "clave-valor" dicho valor no sea tambien una clave de algo mas.
    '''
    col = ['column','value']
    df = pd.DataFrame(list(dict_.items()),columns = col)
    
    df2=df.T
    header=df2.iloc[0]
    df2=df2[1:]
    df2.columns=header
    
    return df2

def get_df_from_response(res,cols):
    '''
    funcion que obtiene todos los diccionarios de la respuesta de la api y los concatena en un solo data frame
    con las columnas que se le pasen. Comprueba que todas las columnas existan si no, no lo concatena.
    '''
    my_dict = res[1][0]
    df = simple_dict2df(my_dict)
    df_total = df.drop(['value'])[cols]

    for dict_ in res[1]:
        df = simple_dict2df(dict_)
        if set(cols).issubset(df.columns):
            df = df[cols]
            df_total = pd.concat([df_total,df])
            
    return df_total

obtenemos los datos de las estaciones actualizados y los metemos en un data frame. Cogemos solo las columnas que nos interesan: Idema (id de la estacion que toma los datos), longitud, latitud, temperatura, temperatura maxima, temperatura minima y ubicacion (descripcion del lugar donde se toman los datos)

In [5]:
respond_weather_actual_all = call_data(url_weather_actual_all)

cols = ['idema','lon','lat','ta','tamin','tamax','ubi']
df_ta_actuals = get_df_from_response(respond_weather_actual_all,cols)

display(df_ta_actuals.sample(5))

column,idema,lon,lat,ta,tamin,tamax,ubi
value,1279X,-6.230729,43.425,7.5,7.4,7.5,CAMUÑO
value,5515X,-3.59555,37.18972,7.9,7.7,7.9,GRANADA-CARTUJA
value,7172X,-1.449172,38.041386,14.5,14.5,14.9,MULA (P. BOMBEROS - AUT.)
value,5598X,-4.547788,37.23093,16.3,16.2,16.5,BENAMEJÍ-ALCACHOFARES ALTOS
value,1014A,-1.792116,43.357075,11.9,11.7,13.6,SAN SEBASTIÁN/FUENTERRABIA


Calculamos una media de las tres medidas (ta, tamin y tamax) para idema, lon, lat y ubi ya que pueden venir mas de una row para las últimas 24h dependiendo de la estación.

In [6]:
df_ta_actuals = df_ta_actuals.groupby(['idema','lon','lat','ubi']).mean().reset_index()

display(df_ta_actuals.sample(5))

column,idema,lon,lat,ubi,ta,tamin,tamax
206,2512Y,-5.018689,40.882130,RIVILLA DE BARAJAS,8.047826,7.726087,8.434783
66,1082,-2.906304,43.297950,BILBAO/AEROPUERTO,12.333333,11.895833,13.133333
281,3319D,-5.012748,40.339664,PUERTO EL PICO,3.969565,3.673913,4.530435
739,B986,1.463528,38.693710,FORMENTERA,12.595833,12.166667,13.045833
549,8325X,-0.371947,39.184723,POLINYA,9.937500,9.612500,10.354167


Obtenemos los datos de la relacion de cada estacion y su provincia asociada

In [7]:
respond_all_station = call_data(url_allstations)

cols = ['indicativo','nombre','provincia']
df_provinces = get_df_from_response(respond_all_station,cols)

display(df_provinces.sample(5))

column,indicativo,nombre,provincia
value,2030,SORIA,SORIA
value,3266A,PUERTO ALTO DEL LEÓN,MADRID
value,1207U,"GIJÓN, CAMPUS",ASTURIAS
value,0252D,ARENYS DE MAR,BARCELONA
value,5910,"ROTA, BASE NAVAL",CADIZ


Juntamos los dos df en uno para tener disponible la provincia, y poder filtrar los datos a pintar (todos va muy lento)

In [8]:
df_ta_provinces = pd.merge(left=df_ta_actuals, right=df_provinces, how='left',
                        left_on='idema', right_on='indicativo')

display(df_ta_provinces.sample(5))

column,idema,lon,lat,ubi,ta,tamin,tamax,indicativo,nombre,provincia
620,9344C,-2.292964,41.216473,ARCOS DE JALÓN-COEX,8.778261,8.456522,9.126087,NaN,NaN,NaN
14,0171X,1.617778,41.578888,IGUALADA,7.954167,7.375000,8.670833,NaN,NaN,NaN
460,6106X,-4.748339,37.029446,ANTEQUERA-BOBADILLA,11.904167,11.545833,12.316667,6106X,ANTEQUERA,MALAGA
447,6001,-5.598830,36.013985,TARIFA,12.720833,12.454167,12.970833,6001,TARIFA,CADIZ
757,C314Z,-17.249166,28.106943,VALLEHERMOSO-ALTO IGUALERO,5.526087,5.269565,5.856522,NaN,NaN,NaN


## Selección de la zona de análisis

Elija una provincia para analizar

In [9]:
province = input('¿Qué Provincia quieres analizar? ').upper()

¿Qué Provincia quieres analizar? madrid


Filtramos el Set de datos por la provincia seleccionada

In [10]:
df_to_map = df_ta_provinces[df_ta_provinces.provincia == province]

display(df_to_map.head())

column,idema,lon,lat,ubi,ta,tamin,tamax,indicativo,nombre,provincia
199,2462,-4.010556,40.793056,NAVACERRADA PUERTO,1.825000,1.475000,2.204167,2462,PUERTO DE NAVACERRADA,MADRID
256,3100B,-3.546025,40.067140,ARANJUEZ,8.662500,8.229167,9.258333,3100B,ARANJUEZ,MADRID
259,3110C,-3.613809,41.006985,BUITRAGO,6.791667,6.425000,7.366667,3110C,BUITRAGO DEL LOZOYA,MADRID
260,3111D,-3.580467,41.135685,SOMOSIERRA,3.762500,3.370833,4.258333,3111D,SOMOSIERRA,MADRID
263,3129,-3.555593,40.466560,MADRID/BARAJAS,8.920833,8.516667,9.504167,3129,MADRID AEROPUERTO,MADRID


Obtenemos la media y los baremos con los que vamos a analizar la zona seleccionada

In [11]:
avg_ta = df_to_map["ta"].mean()
avg_tamax = df_to_map["tamax"].mean()
avg_tamin =df_to_map["tamin"].mean()
avg_ta_up = avg_ta + (avg_tamax - avg_ta)/1.05
avg_ta_down = avg_ta - (avg_ta - avg_tamin)/1.05
print(avg_ta_down, avg_ta, avg_ta_up)

6.283308531746032 6.6578125 7.145907738095239


Graficamos la solucion en un mapa donde nos indiquen las medidas

In [12]:
# crear mapa

map_=folium.Map(location=[29.04824673674675, -13.604344913228068],  # la posicion
                tiles='openstreetmap',            # tipo de mapa, theme
                zoom_start=7)                    # zoom inicial



In [25]:
for index, row in df_to_map.iterrows(): # método para iterar por los dataframes
    
    place_msg = row['nombre'] + "--> "
    ta_msg = " Tª: " + str(round(row['ta'],1)) + " Cº."
    tamax_msg = " Tª Max: " + str(round(row['tamax'],1)) + " Cº."
    tamin_msg =" Tª Min: " + str(round(row['tamin'],1)) + " Cº."
    tooltip_message = place_msg + ta_msg + tamax_msg + tamin_msg
    
    if row['ta'] < avg_ta_down:
    
        folium.Marker([row["lat"], row['lon']],
                      radius=10, 
                      tooltip = tooltip_message,
                      icon=folium.Icon(icon = "cloud", color = "blue")).add_to(map_)
    elif row['ta']  < avg_ta_up:
        folium.Marker([row["lat"], row['lon']],
                      radius=10, 
                      tooltip = tooltip_message,
                      icon=folium.Icon(icon = "cloud", color = "orange")).add_to(map_)
    else:
        folium.Marker([row["lat"], row['lon']],
                      radius=10, 
                      tooltip = tooltip_message,
                      icon=folium.Icon(icon = "cloud", color = "red")).add_to(map_)
        
        

In [24]:
map_